<h1>Homework 04 - Applied ML</h1>

Importation of libraries

In [20]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn import metrics
import matplotlib.pyplot as plt

Importation of the datas

In [2]:
filename = os.path.join('data','CrowdstormingDataJuly1st.csv') 
df = pd.read_csv(filename)
df.head(2)

,playerShort,player,club,leagueCountry,birthday,height,weight,position,games,victories,...,rater2,refNum,refCountry,Alpha_3,meanIAT,nIAT,seIAT,meanExp,nExp,seExp
0,lucas-wilchez,Lucas Wilchez,Real Zaragoza,Spain,31.08.1983,177.0,72.0,Attacking Midfielder,1,0,...,0.50,1,1,GRC,0.326391,712.0,0.000564,0.396000,750.0,0.002696
1,john-utaka,John Utaka,Montpellier HSC,France,08.01.1982,179.0,82.0,Right Winger,1,0,...,0.75,2,2,ZMB,0.203375,40.0,0.010875,-0.204082,49.0,0.061504


<h2>Data exploration</h2>

In [3]:
print('Number of dyads (rows in dataframe): ', len(df))
print('Total number of interactions between a referee and a player (nb of games): ', sum(df.games))
print('Mean number of games for a dyad: ', np.mean(df['games']))

Number of dyads (rows in dataframe):  146028
Total number of interactions between a referee and a player (nb of games):  426572
Mean number of games for a dyad:  2.921165803818446


## Data cleaning / setup

In [4]:
print("Number of rows in dataframe: ", len(df))

Number of rows in dataframe:  146028


<b>Removing raters</b>: We decided to remove row where the two rates were significantly different or if any of the rates were absent (Nan value).

In [5]:
cleandf = df.copy()

## Removing null values in raters
cleandf = cleandf[cleandf["rater1"].notnull() & cleandf["rater2"].notnull()]

## Removing all rows where the difference between the two raters is larger than 0.25
cleandf['difference'] = abs(cleandf.rater1 - cleandf.rater2)
cleandf = cleandf[cleandf['difference'] <= 0.25]
cleandf.drop('difference', axis =1, inplace=True)

print("Number of rows in the cleaned dataframe: ", len(cleandf))
print("Number of rows removed: ", (len(df)-len(cleandf)))

Number of rows in the cleaned dataframe:  124457
Number of rows removed:  21571


<b>Skin tone</b>: Then we decide to take the skin tone as the mean between the two raters. This is the value that will be predicted later.

In [6]:
cleandf["meanSkinTone"] = abs(cleandf["rater1"] - cleandf["rater2"] ) / 2
# cols = cleandf.columns.tolist()
# cols = cols[-1:] + cols[:-1]
# cleandf = cleandf[cols]
cleandf.head(2)

,playerShort,player,club,leagueCountry,birthday,height,weight,position,games,victories,...,refNum,refCountry,Alpha_3,meanIAT,nIAT,seIAT,meanExp,nExp,seExp,meanSkinTone
0,lucas-wilchez,Lucas Wilchez,Real Zaragoza,Spain,31.08.1983,177.0,72.0,Attacking Midfielder,1,0,...,1,1,GRC,0.326391,712.0,0.000564,0.396000,750.0,0.002696,0.125
1,john-utaka,John Utaka,Montpellier HSC,France,08.01.1982,179.0,82.0,Right Winger,1,0,...,2,2,ZMB,0.203375,40.0,0.010875,-0.204082,49.0,0.061504,0.000


<b>Birthday date</b>: As the classifier can not understand date, we decided to change birthday date in seconds. It seemed important for us to keep the birthday date, as it could help predict the color skin if there were more people from a certain demography that played during some years.

In [7]:
def time_to_seconds(t):
    seconds = (pd.to_datetime(t) - datetime.datetime(1970, 1,1)).total_seconds()
    return int(seconds)

cleandf.birthday = cleandf.birthday.apply(time_to_seconds)
cleandf.head(2)

,playerShort,player,club,leagueCountry,birthday,height,weight,position,games,victories,...,refNum,refCountry,Alpha_3,meanIAT,nIAT,seIAT,meanExp,nExp,seExp,meanSkinTone
0,lucas-wilchez,Lucas Wilchez,Real Zaragoza,Spain,431136000,177.0,72.0,Attacking Midfielder,1,0,...,1,1,GRC,0.326391,712.0,0.000564,0.396000,750.0,0.002696,0.125
1,john-utaka,John Utaka,Montpellier HSC,France,397008000,179.0,82.0,Right Winger,1,0,...,2,2,ZMB,0.203375,40.0,0.010875,-0.204082,49.0,0.061504,0.000


<b>Dummy variables</b>: We noticed that a lot of the columns could not be used in the Random forest as they are non-numerical. As most of these features can be seen as categorical variables, we decided to make dummy variables with them.

In [8]:
print("The number of different positions is", cleandf["position"].unique().size)
print("The number of different clubs is",cleandf["club"].unique().size)
print("The number of different league countries is",cleandf["leagueCountry"].unique().size)
print("The number of different referee countries is",cleandf["Alpha_3"].unique().size)

The number of different positions is 13
The number of different clubs is 97
The number of different league countries is 4
The number of different referee countries is 160


Out of these datas, we decided to remove the "referee country" (Alpha_3) and to make dummy variables with the 3 other categories. We decided to remove the referee country because there were a lot of them and it seemed it would induce more error and overfitting to our classifier than it would help it.

In [9]:
dummydf = pd.get_dummies(cleandf, prefix=None, prefix_sep='_', dummy_na=False, columns=["position"], sparse=False, drop_first=False)
dummydf = pd.get_dummies(dummydf, prefix=None, prefix_sep='_', dummy_na=False, columns=["club"], sparse=False, drop_first=False)
dummydf = pd.get_dummies(dummydf, prefix=None, prefix_sep='_', dummy_na=False, columns=["leagueCountry"], sparse=False, drop_first=False)
dummydf.drop("Alpha_3", axis = 1, inplace = True)

In [10]:
print("Number of columns in the previous dataframe: ",cleandf.columns.size)
print("Number of columns in the new dataframe with dummy variables: ",dummydf.columns.size)

Number of columns in the previous dataframe:  29
Number of columns in the new dataframe with dummy variables:  138


<b>Removing useless columns</b>: For the classifier, there are some columns that it makes no sense to use. These columns are the player name (and short name), the photo and the initial ratings (of rater 1 and 2). Therefore we will drop them.

In [31]:
usedf = dummydf.drop(['playerShort', 'photoID', 'rater1', 'rater2']  , axis=1)

<b>Replacing Nan</b>: We realize that the dataframe still have some NaN values. We decided to substitute every NaN with the mean of their column.

In [32]:
usedf.isnull().values.any()

True

In [33]:
# For each column, if there is any NaN value, we compute the mean and replace the NaN values with it.
for i in range(len(usedf.columns)):
    if (usedf[usedf.columns[i]].isnull().values.any()):
        mean = np.mean(usedf[usedf.columns[i]])
        usedf[usedf.columns[i]].fillna(mean, inplace = True)
        
usedf.isnull().values.any()

False

In [34]:
usedf.head(3)

,player,birthday,height,weight,games,victories,ties,defeats,goals,yellowCards,...,club_Werder Bremen,club_West Bromwich Albion,club_West Ham United,club_Wigan Athletic,club_Wolverhampton Wanderers,club_Évian Thonon Gaillard,leagueCountry_England,leagueCountry_France,leagueCountry_Germany,leagueCountry_Spain
0,Lucas Wilchez,431136000,177.0,72.0,1,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,John Utaka,397008000,179.0,82.0,1,0,0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,Aaron Hughes,303177600,182.0,71.0,1,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


## Classifier

Here we will show how we classify the data using random forest.

In [35]:
def executingRandomForest(X, y, Xpd):
    # Creating kfolds
    once = False
    kf = KFold(n_splits=5, shuffle=True, random_state=1)

    #Iterating for kfold validation
    for train_index, test_index in kf.split(X):
        print("TRAIN:", train_index.size, "TEST:", test_index.size)

        ## Making indices
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        #Make classifier
        rf = RandomForestClassifier(n_jobs = 4)

        #Train the classifier
        clf = rf.fit(X_train, y_train)

        # Make prediction for testing data with the classifier
        y_pred = clf.predict(X_test)

        #Computing error
        rf_train_score = metrics.accuracy_score(y_train, clf.predict(X_train))
        rf_test_score = metrics.accuracy_score(y_test, y_pred)
        print('   Training score: {:.5f}'.format(rf_train_score))
        print('   Testing score: {:.5f}'.format(rf_test_score))


        # Features importance score
        if(once != True):
            once = True
            importances = rf.feature_importances_
            std = np.std([tree.feature_importances_ for tree in rf.estimators_],
                         axis=0)
            indices = np.argsort(importances)[::-1]


    # Plot the feature importances of the forest
    plt.figure()
    plt.title("Feature importances")
    plt.bar(range(X.shape[1]), importances[indices],
               color="r", yerr=std[indices], align="center")
    plt.xticks(range(X.shape[1]), importances[indices])
    plt.xlim([-1, X.shape[1]])
    plt.show()

    # Print the feature ranking
    print("Feature ranking (first 20):")
    for f in range(20):
        print("%d. feature %s (%f)" % (f + 1, Xpd.columns[f], importances[indices[f]]))


Making X (the features to use in the classifier) and Y (the value to predict). Executing the random forest algorithm on them

In [36]:
def makeMatrixForClassifier(df):
    y = np.asarray(df["meanSkinTone"].values, dtype="|S6")
    Xpd = df.drop("meanSkinTone", axis = 1)
    X = Xpd.as_matrix()
    return (X, y, Xpd)

In [37]:
(X, y, Xpd) = makeMatrixForClassifier(usedf)
executingRandomForest(X, y, Xpd)

TRAIN: 99565 TEST: 24892


ValueError: could not convert string to float: 'Zdeněk Pospěch'

We can observe that we overfit greatly. Ploting the features importance show....

In [ ]:
notOverfittingMaybedf = usedf.drop(['birthday', 'height', 'weight']  , axis=1)
(X, y, Xpd) = makeMatrixForClassifier(notOverfittingMaybedf)
executingRandomForest(X, y, Xpd)

In [ ]:
notOverfittingMaybedf = usedf.drop(['birthday', 'height', 'weight', 'games', 'victories', 'ties', 'defeats', 'goals']  , axis=1)
(X, y, Xpd) = makeMatrixForClassifier(notOverfittingMaybedf)
executingRandomForest(X, y, Xpd)

## Aggregate the referee info by socker player

In [ ]:
referee_info_df = usedf.drop(['meanIAT', 'nIAT', 'seIAT', 'meanExp', 'nExp', 'seExp', 'refCountry', 'refNum'], axis = 1)
referee_info_df.head()

In [ ]:
by_group_player = list(referee_info_df.groupby('playerShort'))

In [38]:
from functools import reduce

sum_func = lambda x, y: x+y
def accumulate(series):
    return reduce(sum_func, series)

In [39]:
acc_columns = ['games', 'victories', 'ties', 'defeats', 'goals', 'yellowCards', 'yellowReds', 'redCards']

In [40]:
unique_player_data = []
for player_name, data in by_group_player:
    unique_player_data.append((player_name, {column : accumulate(data[column].tolist()) for column in acc_columns}))

NameError: name 'by_group_player' is not defined

In [41]:
len(unique_player_data)

0